In [51]:
import os 
import json
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from typing import Optional
from autogen_agentchat.ui import Console
from autogen_core.models import UserMessage
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from typing import Optional
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()
os.environ["HF_TOKEN"]= os.getenv("HF_TOKEN")

In [ ]:
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

# --- STEP 1: Load JSON file ---
with open("heartbeat_report.json", "r") as f:
    report_data = json.load(f)

# Convert JSON into a pretty text block for retrieval
report_text = json.dumps(report_data, indent=2)

# --- STEP 2: Turn into LangChain Documents ---
docs = [Document(page_content=report_text, metadata={"source": "heartbeat_report"})]

# Split text (if large JSON)
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
doc_chunks = splitter.split_documents(docs)

# --- STEP 3: Build VectorStore ---
embeddings = embeddings 
vectorstore = FAISS.from_documents(doc_chunks, embeddings)
retriever = vectorstore.as_retriever()


def heart_retriever_tool(query: str, top_k: Optional[int] = 4) -> list[Document]:
    """
    Retrieve information from the latest heartbeat_report.json.
    Use this tool to retrieve information from the latest heartbeat_report.json.
    It contains:
    - Classification (Normal, Murmur, Artifact)
    - Signal processing results (BPM, HRV, SNR, etc.)
    - Energy distributions, peaks, and frequency analysis.
    Call this tool whenever a user asks about the analysis results.
    

    Args:
        query (str): The user query about the heartbeat analysis results.
        top_k (Optional[int]): Number of chunks to retrieve. Default = 4.

    Returns:
        list[Document]: Retrieved document chunks containing the most relevant info.
    """
    return retriever.get_relevant_documents(query)[:top_k]


In [ ]:
# Assuming your Ollama server is running locally on port 11434.
ollama_model_client = OllamaChatCompletionClient(model="llama3.2")

agentd = AssistantAgent(
    name="HeartbeatAnalysisAgent",
    model_client=ollama_model_client,
    tools=[heart_retriever_tool],
    description="An agent that answers questions about heartbeat audio analysis results based on classification and signal processing metrics.",
    system_message="""
You are HeartbeatAnalysisAgent, an expert assistant for analyzing heartbeat recordings. 
You have access to pre-computed JSON reports that contain the following metrics:
- Classification results (Artifact, Murmur, Normal)
- BPM (beats per minute) and rhythm regularity
- HRV (Heart Rate Variability) metrics: mean BPM, SDNN, RMSSD, CV
- SNR (signal-to-noise ratio)
- Energy distribution (how signal power is distributed across frequencies)
- S1/S2 amplitude ratio
- Extra peak detection (murmurs, gallops, abnormal beats)
- Irregular spacing statistics
- Frequency band energy (150–500 Hz band for murmurs)
- Visual outputs (waveform, spectrogram, histograms)

Your role:
- Answer user questions using ONLY the data provided in the JSON report or general definitions of the above metrics.
- If asked about medical interpretation, provide general information only (e.g., what a high HRV usually means), not medical advice.
- If a query is unrelated to heartbeat analysis, classification, or signal processing, reply with: "I don't know."
- Do not make up or hallucinate answers.
- Keep explanations clear and concise. 
- When a user explicitly asks you to stop or when a conversation session is complete, respond with "TERMINATE".
""",
reflect_on_tool_use=True
)



In [50]:
res = await Console(agentd.run_stream(task = "What was my last question"))
res.messages[-1].content

---------- TextMessage (user) ----------
What was my last question
---------- ToolCallRequestEvent (HeartbeatAnalysisAgent) ----------
[FunctionCall(id='2', arguments='{"query": "last question about BPM and HRV explanation", "top_k": null}', name='heart_retriever_tool')]
---------- ToolCallExecutionEvent (HeartbeatAnalysisAgent) ----------
[FunctionExecutionResult(content='[Document(id=\'7f94d01b-dec4-4ba9-ad93-48eff0cdde38\', metadata={\'source\': \'heartbeat_report\'}, page_content=\'{\\n  "file": "C:\\\\\\\\Users\\\\\\\\djadh\\\\\\\\AppData\\\\\\\\Local\\\\\\\\Temp\\\\\\\\tmp3d3n5yid.wav",\\n  "fs": 2000,\\n  "duration_s": 2.236,\\n  "beats_detected": 8,\\n  "bpm": 209.1112770724421,\\n  "classification_result": "murmur",\\n  "hrv": {\\n    "mean_IBI_ms": 286.92857142857144,\\n    "median_IBI_ms": 276.5,\\n    "bpm_mean": 209.1112770724421,\\n    "SDNN_ms": 23.518483106251722,\\n    "RMSSD_ms": 40.219709098898264,\'), Document(id=\'0d3b6d02-4d06-4e06-a366-4918687e9d7f\', metadata={\

"It seems like your last question was about adding two numbers, but I'm a heartbeat analysis assistant, and I don't have the capability to perform mathematical operations or provide code for basic arithmetic like addition. If you need help with something else related to heartbeat analysis, feel free to ask!"

## report

In [1]:
import json
from datetime import datetime

def generate_hospital_report(report_path: str):
    # Load JSON
    with open(report_path, "r") as f:
        data = json.load(f)
    
    # Extract fields
    classification = data.get("classification", "Unknown")
    duration = data.get("duration_s", 0)
    beats = data.get("beats_detected", 0)
    bpm = data.get("bpm", 0)
    snr = data.get("SNR_dB", 0)
    
    hrv = data.get("hrv", {})
    energy = data.get("energy", {})
    s1s2 = data.get("S1S2", {})
    extra_peaks = data.get("extra_peaks", {})
    band_energy = data.get("150_500Hz_band", {})
    
    # Demo patient details (predefined)
    patient_name = "Demo Patient"
    patient_age = 45
    patient_gender = "Male"
    
    # Generate report
    print("=============================================")
    print("          HEART SOUND ANALYSIS REPORT        ")
    print("=============================================")
    print(f"Patient: {patient_name}   Age: {patient_age}   Gender: {patient_gender}")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("---------------------------------------------")
    
    print(f"Recording Duration: {duration:.2f} s")
    print(f"Beats Detected: {beats}")
    print(f"Average HR: {bpm:.1f} bpm")
    print(f"Signal-to-Noise Ratio (SNR): {snr:.2f} dB")
    print("---------------------------------------------")
    
    print("Classification Result:")
    print(f" - Category: {classification}")
    print("")
    
    print("Heart Rate Variability (HRV):")
    print(f" - Mean IBI: {hrv.get('mean_IBI_ms',0):.1f} ms")
    print(f" - SDNN: {hrv.get('SDNN_ms',0):.2f} ms")
    print(f" - RMSSD: {hrv.get('RMSSD_ms',0):.2f} ms")
    print(f" - pNN50: {hrv.get('pNN50_pct',0):.2f} %")
    print("")
    
    print("Valve Sounds (S1 / S2):")
    print(f" - S1 Mean: {s1s2.get('S1_mean',0):.3f}")
    print(f" - S2 Mean: {s1s2.get('S2_mean',0):.3f}")
    print(f" - S1/S2 Ratio: {s1s2.get('S1_to_S2_ratio',0):.2f}")
    print("")
    
    print("Spectral / Frequency Analysis:")
    print(f" - Energy <200 Hz: {energy.get('pct_energy_below_200Hz',0):.2f} %")
    print(f" - 150–500 Hz Band Energy: {band_energy.get('band_energy_pct',0):.2f} %")
    print(f" - Spectral Centroid: {energy.get('spectral_centroid_hz',0):.2f} Hz")
    print("")
    
    print("Extra Peaks / Irregularities:")
    print(f" - Cycles with Extra Peaks: {extra_peaks.get('cycles_with_extra_peaks',0)}")
    print("")
    
    print("Impression:")
    if classification == "Normal":
        print("Regular rhythm, adequate signal quality, no murmurs detected.")
    elif classification == "Murmur":
        print("Possible murmur detected. Recommend clinical follow-up.")
    else:
        print("Signal quality insufficient or inconclusive result.")
    
    print("---------------------------------------------")
    print("Disclaimer: This report is generated by AI analysis for demo purposes only.")
    print("Not intended for clinical diagnosis.")
    print("=============================================")


# Example usage
generate_hospital_report("heartbeat_report.json")


          HEART SOUND ANALYSIS REPORT        
Patient: Demo Patient   Age: 45   Gender: Male
Date: 2025-09-12 18:54:01
---------------------------------------------
Recording Duration: 2.24 s
Beats Detected: 8
Average HR: 209.1 bpm
Signal-to-Noise Ratio (SNR): 9.52 dB
---------------------------------------------
Classification Result:
 - Category: Normal

Heart Rate Variability (HRV):
 - Mean IBI: 286.9 ms
 - SDNN: 23.52 ms
 - RMSSD: 40.22 ms
 - pNN50: 16.67 %

Valve Sounds (S1 / S2):
 - S1 Mean: 0.075
 - S2 Mean: 0.083
 - S1/S2 Ratio: 0.91

Spectral / Frequency Analysis:
 - Energy <200 Hz: 95.81 %
 - 150–500 Hz Band Energy: 6.49 %
 - Spectral Centroid: 76.84 Hz

Extra Peaks / Irregularities:
 - Cycles with Extra Peaks: 0

Impression:
Regular rhythm, adequate signal quality, no murmurs detected.
---------------------------------------------
Disclaimer: This report is generated by AI analysis for demo purposes only.
Not intended for clinical diagnosis.
